# Example of training Graph Neural Networks (GNNs)

GRB provides easy-to-use APIs to train GNNs, facilitating the entire process from loading graph data, building GNN models, to evaluation and inference. Here is an example of training GCNs (Graph Convolution Networks).

Contents
- [Load Dataset](#Load-Dataset)
- [Build Model](#Build-Model)
- [Training](#Training)
- [Inference](#Inference)
- [Evaluation](#Evaluation)

In [7]:
import os
import torch
import grb.utils as utils

## Load Dataset

GRB datasets are named by the prefix *grb-*. There are four *mode* ('easy', 'medium', 'hard', 'full') for test set, representing different average degrees of test nodes, thus different difficulty for attacking them. The node features are processed by *arctan* normalization (first standardization then arctan function), which makes node features fall in the same scale.

In [8]:
from grb.dataset import Dataset

dataset_name = 'grb-cora'
dataset = Dataset(name=dataset_name, 
                  data_dir="../data/",
                  mode='full',
                  feat_norm='arctan')

Dataset 'grb-cora' loaded.
    Number of nodes: 2680
    Number of edges: 5148
    Number of features: 302
    Number of classes: 7
    Number of train samples: 1608
    Number of val samples: 268
    Number of test samples: 804
    Dataset mode: full
    Feature range: [-0.9406, 0.9430]


## Build Model

GRB supports models based on pure Pytorch, CogDL or DGL. The following is an example of GCN implemented by pure Pytorch. Other models can be found in ``grb/model/torch``, ``grb/model/cogdl``, or ``grb/model/dgl``.

### GCN

In [32]:
from grb.model.torch import GCN
from grb.utils.normalize import GCNAdjNorm

model_name = "gcn"
model = GCN(in_features=dataset.num_features,
            out_features=dataset.num_classes,
            hidden_features=[64, 64], 
            adj_norm_func=GCNAdjNorm,
            layer_norm=False,
            residual=False,
            dropout=0.5)
print("Number of parameters: {}.".format(utils.get_num_params(model)))
print(model)

Number of parameters: 24007.
GCN(
  (layers): ModuleList(
    (0): GCNConv(
      (linear): Linear(in_features=302, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): GCNConv(
      (linear): Linear(in_features=64, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (2): GCNConv(
      (linear): Linear(in_features=64, out_features=7, bias=True)
    )
  )
)


### APPNP

In [10]:
from grb.model.torch import APPNP
from grb.utils.normalize import GCNAdjNorm

model_name = "appnp"
model = APPNP(in_features=dataset.num_features,
              out_features=dataset.num_classes,
              hidden_features=[64, 64], 
              adj_norm_func=GCNAdjNorm,
              layer_norm=False,
              edge_drop=0.1,
              alpha=0.01,
              k=3,
              dropout=0.5)
print("Number of parameters: {}.".format(utils.get_num_params(model)))
print(model)

Number of parameters: 24007.
APPNP(
  (layers): ModuleList(
    (0): Linear(in_features=302, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=7, bias=True)
  )
  (edge_dropout): SparseEdgeDrop()
  (dropout): Dropout(p=0.5, inplace=False)
)


## Training

GRB provides ``grb.utils.trainer`` that facilitates the training process of GNNs. The training mode can be chosen from ``inductive`` or ``transductive``. In the inductive mode, only train nodes can be seen during training, train+val nodes can be seen during validation, train+val+test nodes can be seen during testing. In the transductive mode, all nodes are available for each process.  

In [33]:
save_dir = "./saved_modes/{}/{}".format(dataset_name, model_name)
save_name = "model.pt"
device = "cuda:0"
feat_norm = None
train_mode = "transductive"

In [34]:
from grb.utils.trainer import Trainer

trainer = Trainer(dataset=dataset, 
                  optimizer=torch.optim.Adam(model.parameters(), lr=0.01),
                  loss=torch.nn.functional.nll_loss,
                  lr_scheduler=False,
                  early_stop=True,
                  early_stop_patience=500,
                  feat_norm=feat_norm,
                  device=device)

In [35]:
trainer.train(model=model, 
              n_epoch=2000,
              eval_every=1,
              save_after=0,
              save_dir=save_dir,
              save_name=save_name,
              train_mode=train_mode)

Epoch 00086 | Train loss 0.6195 | Train score 0.7991 | Val loss 0.5759 | Val score 0.8134:   4%|▎         | 72/2000 [00:00<00:11, 174.86it/s]

Epoch 00051 | Best validation score: 0.7836
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00052 | Best validation score: 0.7910
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00060 | Best validation score: 0.7948
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00061 | Best validation score: 0.7985
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00063 | Best validation score: 0.8022
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00064 | Best validation score: 0.8134
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00065 | Best validation score: 0.8172
Model saved in './saved_modes/grb-cora/gcn/model.pt'.


Epoch 00126 | Train loss 0.4721 | Train score 0.8451 | Val loss 0.4907 | Val score 0.8545:   6%|▋         | 127/2000 [00:00<00:10, 175.68it/s]

Epoch 00092 | Best validation score: 0.8209
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00093 | Best validation score: 0.8246
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00096 | Best validation score: 0.8284
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00097 | Best validation score: 0.8321
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00102 | Best validation score: 0.8358
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00104 | Best validation score: 0.8396
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00118 | Best validation score: 0.8433
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00123 | Best validation score: 0.8470
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00125 | Best validation score: 0.8507
Model saved in './saved_modes/grb-cora/gcn/model.pt'.
Epoch 00126 | Best validation score: 0.8545
Model saved in './saved_modes/grb-cora/gcn/model.pt'.


Epoch 00740 | Train loss 0.1370 | Train score 0.9471 | Val loss 0.6931 | Val score 0.8060:  37%|███▋      | 741/2000 [00:04<00:07, 168.99it/s]

Training early stopped. Best validation score: 0.8545
Model saved in './saved_modes/grb-cora/gcn/early_stopped_model.pt'.


## Inference

In [36]:
model = torch.load(os.path.join(save_dir, save_name))
model = model.to(device)
model.eval()

GCN(
  (layers): ModuleList(
    (0): GCNConv(
      (linear): Linear(in_features=302, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): GCNConv(
      (linear): Linear(in_features=64, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (2): GCNConv(
      (linear): Linear(in_features=64, out_features=7, bias=True)
    )
  )
)

In [37]:
# by trainer
pred = trainer.inference(model)
print(pred, pred.shape)

tensor([[-1.8770e+00, -3.8322e-01, -2.3015e+00,  ..., -1.1700e+00,
         -3.7822e+00, -3.8087e+00],
        [-2.6523e-01, -3.9858e-01, -1.3438e+00,  ...,  7.8287e+00,
         -6.5029e+00, -2.4968e+00],
        [ 7.2530e-01, -2.6292e-01, -1.3161e+00,  ...,  3.5806e+00,
         -6.9180e+00, -2.9982e+00],
        ...,
        [ 1.5502e+00, -1.5170e+00,  1.0556e+00,  ..., -1.5491e+00,
         -5.1555e+00, -2.6115e+00],
        [-1.1753e+00, -6.9723e-03, -2.1175e+00,  ..., -3.6916e+00,
          1.0135e+00, -1.3410e+00],
        [-1.5373e+00,  5.0471e-03, -6.4687e-01,  ..., -1.6436e+00,
         -3.7218e+00, -3.6808e+00]], device='cuda:0', grad_fn=<MmBackward>) torch.Size([2680, 7])


In [38]:
# by utils
pred = utils.inference(model, 
                       features=dataset.features,
                       feat_norm=feat_norm,
                       adj=dataset.adj,
                       adj_norm_func=model.adj_norm_func,
                       device=device)
print(pred, pred.shape)

tensor([[-1.8770e+00, -3.8322e-01, -2.3015e+00,  ..., -1.1700e+00,
         -3.7822e+00, -3.8087e+00],
        [-2.6523e-01, -3.9858e-01, -1.3438e+00,  ...,  7.8287e+00,
         -6.5029e+00, -2.4968e+00],
        [ 7.2530e-01, -2.6292e-01, -1.3161e+00,  ...,  3.5806e+00,
         -6.9180e+00, -2.9982e+00],
        ...,
        [ 1.5502e+00, -1.5170e+00,  1.0556e+00,  ..., -1.5491e+00,
         -5.1555e+00, -2.6115e+00],
        [-1.1753e+00, -6.9723e-03, -2.1175e+00,  ..., -3.6916e+00,
          1.0135e+00, -1.3410e+00],
        [-1.5373e+00,  5.0471e-03, -6.4687e-01,  ..., -1.6436e+00,
         -3.7218e+00, -3.6808e+00]], device='cuda:0', grad_fn=<MmBackward>) torch.Size([2680, 7])


## Evaluation

In [39]:
# by trainer
test_score = trainer.evaluate(model, dataset.test_mask)
print("Test score: {:.4f}".format(test_score))

Test score: 0.8470


In [40]:
# by utils
test_score = utils.evaluate(model, 
                            features=dataset.features,
                            adj=dataset.adj,
                            labels=dataset.labels,
                            feat_norm=feat_norm,
                            adj_norm_func=model.adj_norm_func,
                            mask=dataset.test_mask,
                            device=device)
print("Test score: {:.4f}".format(test_score))

Test score: 0.8470


For further information, please refer to the [GRB Documentation](https://grb.readthedocs.io/en/latest/).